In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import os

import sys
sys.path.append('../')
from src.utils.parse_cas_operon_tab import parse_cas_operon_tab

# Plot the distributions of the Cas-system types found in the archaeal genomes

In [1]:
def plot_distributions(dict):
    """
    Dict containing {
        "type_name": abundance
    }
    """
    pass

def retrieve_cas_system_types_found():
    pass

In [2]:
# ASGARD

In [3]:
# DPANN

In [4]:
# EURY

In [5]:
#  TACK

In [6]:
# using any CRISPR-arrays found, how many spacers are possible in the dataset? For example the histogram on the bottom of the scatterplot is the number shows the distribution of the number of spacers per genome across all genomes in the dataset 

In [7]:
# what is the average length (in nucleotides) of the spacers found? 